# 1.0 — Collecte de données marché (Yahoo Finance)


**Explication :** 
- On les télécharge les données du S&p 500, SPY, VIX, depuis Yehoo Finance ( on utilise la librairie), 
- On normalise les colonnes pour éviter tout problèmes et par la suite on sauvegarde les CSV et les Parquets en Json pour les traiter par la suite.



In [1]:

from datetime import datetime, timezone
from pathlib import Path
import json
import pandas as pd
import yfinance as yf

TICKERS = {"SPX_INDEX": "^GSPC", "SPY_ETF": "SPY", "VIX_INDEX": "^VIX"}
START_DATE = "1993-01-29"
END_DATE = None  # None = jusqu'à aujourd'hui
DATA_DIR = Path("data/raw")
DATA_DIR.mkdir(parents=True, exist_ok=True)


## 1.1 Fonction "fetch" — téléchargement & normalisation

**Explication :**
1) On télécharge les données, et on utilise auto_adjust=True pour la colonnes du prix de clotures pour avoir un pric corrigé des splits.  
2) Aplatis les colonnes et enleve les suffixes ticker (code produit d'un actif financier ).
3) Force la présence de la variable "date" et adj_close.
4) On trie les lignes par date chronologie.
5) On supprime les données "NaN"

**Pourquoi nous avons fait cela :** 
1) On utilise auto_adjust=True pour ensuite faciliter le calcul des rendements mais aussi éviter de gérer manuellement la colonne "close", cela permet de garantir que le prix utilisé reflete bien la réalité économique (données à jours).
2) "Aplatir les colonnes + supprimer les suffixes" -> Permet de rendre le dataset homogène et évite d'avoir des noms de colonnes différents selon l'actif.
3) "Forcer la présence de date et adj_close" -> Assure d'avoir une structure standardisé pour les futures traitements.
4) "Trier par date" -> garantit une série temporelle strictement chronologique.
5) "Supprimer les NaN" -> Empeche la propagation d'erreurs statistiques 



In [ ]:
def fetch(ticker: str, start: str, end: str | None) -> pd.DataFrame:

    df = yf.download(
        ticker, start=start, end=end,
        auto_adjust=True,        
        group_by="column",
        actions=False,
        progress=False,
        threads=True,            
    )
    if df is None or df.empty:
        raise RuntimeError(f"Aucune donnée pour {ticker}")


    
    df = df.dropna(how="all").reset_index()
    def _flatten_cols(cols):
        out = []
        for c in cols:
            if isinstance(c, tuple):
                s = "_".join([str(x) for x in c if x not in (None, "", "None")])
            else:
                s = str(c)
            out.append(s.lower().replace(" ", "_"))
        return out
    df.columns = _flatten_cols(df.columns)


    suf = "_" + ticker.lower()
    rename_map = {}
    for c in df.columns:
        if c in ("date", "index"):
            continue
        base = c[:-len(suf)] if c.endswith(suf) else c
        rename_map[c] = base
    df = df.rename(columns=rename_map)

    if "date" not in df.columns and "index" in df.columns:
        df = df.rename(columns={"index": "date"})
    if "adj_close" not in df.columns and "close" in df.columns:
        df["adj_close"] = df["close"]
    if "adj_close" not in df.columns and "close" not in df.columns:
        raise KeyError(f"Pas de 'close'/'adj_close' après normalisation. Colonnes: {list(df.columns)}")

    df = df.sort_values("date").dropna(subset=["adj_close"]).reset_index(drop=True)
    return df


## 1.2 Fonction save — sauvegarde & métadonnées

**Explication :**
1) Sauvegarde le DataFrame en CSV (lisible partout) et en Parquet (format optimisé).
2) Retourne un dictionnaire de métadonnées :
    - nombre de lignes,
    - première et dernière date de la série
    - chemins des fichiers écrits
    - liste des colonnes.

**Pourquoi nous avons fait cela :** 
1) Un a un CSV comme format universel et qu'il soit lisible facilement, et un format "Parquet" qui est un format optimisé (donc rapide)
2) Permet de retourner des métadonnées et permet de vérifier rapidement la cohérence des données


In [ ]:

def save(df: pd.DataFrame, name: str) -> dict:
    p_csv = DATA_DIR / f"{name}.csv"
    p_par = DATA_DIR / f"{name}.parquet"
    df.to_csv(p_csv, index=False)
    df.to_parquet(p_par, index=False)
    return {
        "rows": len(df),
        "first_date": df["date"].iloc[0].strftime("%Y-%m-%d"),
        "last_date": df["date"].iloc[-1].strftime("%Y-%m-%d"),
        "csv": str(p_csv),
        "parquet": str(p_par),
        "columns": df.columns.tolist(),
    }



## 1.3 Bloc principal — exécution & génération des métadonnées

**Explication :**
1) On créer un dictionnaire qui stocke tout les données.
2) Pour chaque ticker on télécharge les donnés via à la fonction "fetch" et on sauvegarde via la fonction "save".
3) On écrit un fichier "metadata.json" et un récapitulatif dans "DATA_DIR"
4) Créer un message message de confirmation en fin de collecte.


**Pourquoi nous avons fait cela :** 
1) Générer le fichier "meta" centralise toutes les informations de traçabilité (source, période, tickers, options).
2) Permet de rendre la collecte scalable (ajouter un actif = 1 ligne dans la config).
3) Permet de faciliter la reproductibilité et le debug.
4) Le print final sert de feedback rapide pour confirmer que la collecte s’est bien déroulée.

In [ ]:
if __name__ == "__main__":
    meta = {
        "collected_at_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "source": "Yahoo Finance via yfinance",
        "universe": ["^GSPC", "SPY", "^VIX"],
        "period": {"start": START_DATE, "end": END_DATE or "today"},
        "datasets": {},
        "notes": [
            "Colonnes aplanies pour éviter MultiIndex.",
            "Prix ajustés (auto_adjust=True) pour des rendements cohérents.",
        ],
    }

    for name, ticker in TICKERS.items():
        print(f"→ Téléchargement {name} ({ticker})")
        df = fetch(ticker, START_DATE, END_DATE)
        meta["datasets"][name] = {"ticker": ticker, **save(df, name.lower())}

    (DATA_DIR / "metadata.json").write_text(
        json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8"
    )
    print("✅ Collecte terminée →", DATA_DIR.resolve())


→ Téléchargement SPX_INDEX (^GSPC)
→ Téléchargement SPY_ETF (SPY)
→ Téléchargement VIX_INDEX (^VIX)
✅ Collecte terminée → C:\Users\antoi\Proton Drive\antoinebuisson000\My files\12 - SKEMA\4 - CARRIERE FINANCE\1 - PROJET FINANCE QUANT\notebook\data\raw


### 1.4 Chargement des données brutes — inspection & métadonnées

**Explication :**
1) Affiche le contenu du dossier data/raw pour vérifier quels fichiers ont été collectés.
2) Lit le fichier metadata.json et le charge en dictionnaire Python m.
3) Affiche la structure des métadonnées (aperçu du contenu).

In [2]:
from pathlib import Path
import json
import pandas as pd

print("Contenu du dossier:", list(Path("data/raw").glob("*")))
m = json.loads(Path("data/raw/metadata.json").read_text(encoding="utf-8"))
m


Contenu du dossier: [WindowsPath('data/raw/metadata.json'), WindowsPath('data/raw/spx_index.csv'), WindowsPath('data/raw/spx_index.parquet'), WindowsPath('data/raw/spy_etf.csv'), WindowsPath('data/raw/spy_etf.parquet'), WindowsPath('data/raw/vix_index.csv'), WindowsPath('data/raw/vix_index.parquet')]


{'collected_at_utc': '2025-09-18T10:30:48+00:00',
 'source': 'Yahoo Finance via yfinance',
 'universe': ['^GSPC', 'SPY', '^VIX'],
 'period': {'start': '1993-01-29', 'end': 'today'},
 'datasets': {'SPX_INDEX': {'ticker': '^GSPC',
   'rows': 8215,
   'first_date': '1993-01-29',
   'last_date': '2025-09-17',
   'csv': 'data\\raw\\spx_index.csv',
   'parquet': 'data\\raw\\spx_index.parquet',
   'columns': ['date', 'close', 'high', 'low', 'open', 'volume', 'adj_close']},
  'SPY_ETF': {'ticker': 'SPY',
   'rows': 8215,
   'first_date': '1993-01-29',
   'last_date': '2025-09-17',
   'csv': 'data\\raw\\spy_etf.csv',
   'parquet': 'data\\raw\\spy_etf.parquet',
   'columns': ['date', 'close', 'high', 'low', 'open', 'volume', 'adj_close']},
  'VIX_INDEX': {'ticker': '^VIX',
   'rows': 8216,
   'first_date': '1993-01-29',
   'last_date': '2025-09-18',
   'csv': 'data\\raw\\vix_index.csv',
   'parquet': 'data\\raw\\vix_index.parquet',
   'columns': ['date',
    'close',
    'high',
    'low',
    '